In [1]:
from board import Board
import numpy as np
import random
import utils
import collections

In [2]:
def get_missing_digits(board, x):
    """
    Given a board and row x, return the set of digits not present in that row
    """
    return set(np.arange(board.max_digit) + 1) - set(board[x][board[x] > 0])

def get_box(board, x, y):
    """
    Returns a copy of the sub-array of the box that (x, y) belongs in and box-index
    """
    box_x_min = board.dim_x * (x // board.dim_x)
    box_x_max = box_x_min + board.dim_x
    box_y_min = board.dim_y * (y // board.dim_y)
    box_y_max = box_y_min + board.dim_y
    box = np.array(board[box_x_min:box_x_max, box_y_min:box_y_max])
    return box,  

def write_random_digits_to_row(board, x, fill):
    unused_digits = get_missing_digits(board, x)
    empty_ys = np.nonzero(board[x] == 0)[0]
    np.random.shuffle(empty_ys)
    
    for empty_y in empty_ys:
        if fill == 0:
            break
        fill_digit = random.sample(unused_digits, 1)[0]
        board.write(x, empty_y, fill_digit)
        unused_digits.remove(fill_digit)
        fill -= 1
    return board

def generate_hidden_single_row(board, x, y, digit, filled=5):
    """
    If want to generate column, just take the transpose
    """
    board = board.copy()
    filled -= np.sum(board[x] > 0)
    
    board.write(x, y, digit)
    board = write_random_digits_to_row(board, x, filled)
    board = board.remove(x, y)
    all_xs = np.arange(board.max_digit) 
    box_y_min, box_y_max = board.get_box_boundaries_y(y)
    empty_ys = np.nonzero(board[x] == 0)[0]
    np.random.shuffle(empty_ys)
    for empty_y in empty_ys:
        if empty_y != y and board.is_candidate(x, empty_y, digit):
            possible_xs = {x_i for x_i in all_xs if board.is_candidate(x_i, empty_y, digit) and x_i != x}
            if box_y_min <= empty_y < box_y_max:
                box_x_min, box_x_max = board.get_box_boundaries_x(x)
                possible_xs -= set(range(box_x_min, box_x_max))
            x2 = random.sample(possible_xs, 1)[0]
            board.write(x2, empty_y, digit)
    return board
    

def generate_hidden_single(board, x, y, digit, house='row', filled=5, ):
    assert house in ['row', 'column', 'box']
    

In [4]:
board = Board(3, 3)
generate_hidden_single_row(board, 1, 2, 3, 0)

array([[0, 0, 0, 0, 0, 0, 3, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 3, 0, 0, 0, 0, 0, 0, 0],
       [3, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 3, 0, 0, 0]], dtype=int8)